<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [145]:
from getpass import getpass
HF_TOKEN = getpass("Enter your Hugging Face token: ")

Enter your Hugging Face token: ··········


In [146]:
from huggingface_hub import login
login(token=HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [147]:
import os, sys, shutil

os.chdir("/content")
current_dir = os.getcwd()
print(f"The current directory is {current_dir}")

if os.path.isdir('ML_GitHub'):
    shutil.rmtree('ML_GitHub')

!git init
!git branch -m main
!git clone https://github.com/claudelepere/ML_GitHub.git
print(f"list /content: {os.listdir(current_dir)}")
print(f"list /content/ML_GitHub: {os.listdir(current_dir + '/ML_GitHub')}")
!ls -la /content/ML_GitHub/datasetHF
os.chdir(current_dir + '/ML_GitHub')

#!pip install fsspec==2024.10.0
!pip install -q transformers datasets
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk('datasetHF')
print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")

The current directory is /content
Reinitialized existing Git repository in /content/.git/
Cloning into 'ML_GitHub'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 65 (delta 29), reused 50 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 9.98 MiB | 23.66 MiB/s, done.
Resolving deltas: 100% (29/29), done.
list /content: ['.config', '.git', 'ML_GitHub', 'sample_data']
list /content/ML_GitHub: ['.git', 'datasetHF', 'README.md', 'Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb']
total 24
drwxr-xr-x 5 root root 4096 Nov  6 21:59 .
drwxr-xr-x 4 root root 4096 Nov  6 21:59 ..
-rw-r--r-- 1 root root   43 Nov  6 21:59 dataset_dict.json
drwxr-xr-x 2 root root 4096 Nov  6 21:59 test
drwxr-xr-x 2 root root 4096 Nov  6 21:59 train
drwxr-xr-x 2 root root 4096 Nov  6 21:59 validation
dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (12

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's check the first example of the training split:

In [148]:
example = dataset['validation'][0]
example

{'id': 141545,
 'text': 'Actiris - Scrum Master / Gestionnaire de Projets Scrum Master, Chef de Projets, .NET Actiris Le service développement d\'Actiris est composé d\'une trentaine de personnes qui travaillent essentiellement sur des projets nouveaux (dossier unique, plateforme pour la gestion des relations entre Actiris et les partenaires, plateforme pour la gestion des relations entre Actiris et les employeurs, mise en place d\'un nouveau CRM, etc.). Votre rôle En tant que Scrum Master/gestionnaire de projets, vous initiez et coordonnez, dans un environnement Agile, des projets dans le cadre du développement de nos nouvelles plateformes core business et vous veillez à leur bonne réalisation dans les délais impartis. Vos responsabilités Vous collectez les informations nécessaires au bon déroulement du projet et êtes capable de discuter d\'un backlog avec le métier ; Vous déterminez les étapes clés, planning et rétro-planning Highlevel sans forcément de requête auprès de l\'équipe de

The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [149]:
labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()

print(len(labels), list(enumerate(labels)))
id2label = {idx:label for idx, label in enumerate(labels)}
print(id2label)
label2id = {label:idx for idx, label in enumerate(labels)}
print(len(labels), labels)

42 [(0, '142'), (1, '146'), (2, '147'), (3, '148'), (4, '149'), (5, '150'), (6, '151'), (7, '152'), (8, '153'), (9, '154'), (10, '155'), (11, '156'), (12, '157'), (13, '158'), (14, '160'), (15, '162'), (16, '165'), (17, '167'), (18, '168'), (19, '169'), (20, '170'), (21, '171'), (22, '173'), (23, '174'), (24, '176'), (25, '356'), (26, '360'), (27, '361'), (28, '362'), (29, '364'), (30, '375'), (31, '376'), (32, '394'), (33, '408'), (34, '409'), (35, '685'), (36, '686'), (37, '689'), (38, '756'), (39, '758'), (40, '760'), (41, '761')]
{0: '142', 1: '146', 2: '147', 3: '148', 4: '149', 5: '150', 6: '151', 7: '152', 8: '153', 9: '154', 10: '155', 11: '156', 12: '157', 13: '158', 14: '160', 15: '162', 16: '165', 17: '167', 18: '168', 19: '169', 20: '170', 21: '171', 22: '173', 23: '174', 24: '176', 25: '356', 26: '360', 27: '361', 28: '362', 29: '364', 30: '375', 31: '376', 32: '394', 33: '408', 34: '409', 35: '685', 36: '686', 37: '689', 38: '756', 39: '758', 40: '760', 41: '761'}
42 ['14

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [150]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# examples and not example because batched=True => examples is a batch
def preprocess_data(examples, indices):
  # take a batch of texts
  text = examples["text"]
  #print("Indices:", indices, "Batch size:", len(text), "Num labels:", len(labels))

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)

  labels_batch = {label: np.zeros(len(text)) for label in labels}                                                # validation: 42 rows: 'label': 18 x 0.0
  #print(f"labels_batch: zeros: {type(labels_batch)} {len(labels_batch)} {labels_batch}")
  #print(f"examples.keys(): {examples.keys()}")
  # examples.keys(): all the column names (keys) in the batch dict "examples"
  # for each k in labels, a new key-value pair k: examples[k] is added to labels_batch
  labels_batch.update({k: [1.0 if val else 0.0 for val in examples[k]] for k in examples.keys() if k in labels}) # validation: 42 rows: 'label': 1.0 or 0.0

  #print(f"labels_batch: updated: {type(labels_batch)} {len(labels_batch)} {labels_batch}")

  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))                                                             # validation: 18 rows 42 columns
  #print(f"labels_matrix:{type(labels_matrix)} {labels_matrix.shape}")
  # fill numpy array
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")                                                                            # labels are sorted
    labels_matrix[:, idx] = labels_batch[label] # for each row, sets the idx-th column of labels_matrix with the values from labels_batch[label]

  #print("First row of labels_matrix:", labels_matrix[0])

  # Add labels to encoding
  encoding["labels"] = labels_matrix.tolist()
  #print(f"encoding['labels']: {encoding['labels']}")


  return encoding



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [151]:
#dataset.map(preprocess_data_ONE, batched=False)

In [152]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names, with_indices=True)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [153]:
example = encoded_dataset['validation'][0]
print(f"example.keys(): {example.keys()}")
print(f"example['input_ids']: {example['input_ids']}")
print(f"example['token_type_ids']: {example['token_type_ids']}")
print(f"example['attention_mask']: {example['attention_mask']}")
print(f"example['labels']: {example['labels']}")

example.keys(): dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
example['input_ids']: [101, 2552, 15735, 2015, 1011, 8040, 6824, 3040, 1013, 16216, 16643, 2239, 20589, 2139, 4013, 15759, 2015, 8040, 6824, 3040, 1010, 10026, 2139, 4013, 15759, 2015, 1010, 1012, 5658, 2552, 15735, 2015, 3393, 2326, 4503, 5051, 3672, 1040, 1005, 2552, 15735, 2015, 9765, 17202, 1040, 1005, 16655, 7990, 18175, 2139, 2711, 5267, 21864, 19817, 12462, 10484, 3372, 29032, 9515, 6216, 3672, 7505, 4078, 4013, 15759, 2015, 25272, 2595, 1006, 9998, 20236, 4310, 1010, 5127, 14192, 2063, 10364, 2474, 16216, 16643, 2239, 4078, 4262, 4372, 7913, 2552, 15735, 2015, 3802, 4649, 2112, 8189, 7442, 2015, 1010, 5127, 14192, 2063, 10364, 2474, 16216, 16643, 2239, 4078, 4262, 4372, 7913, 2552, 15735, 2015, 3802, 4649, 12666, 26744, 1010, 28616, 2063, 4372, 2173, 1040, 1005, 4895, 25272, 13675, 2213, 1010, 4385, 1012, 1007, 1012, 29536, 7913, 2535, 4372, 9092, 2102, 10861, 8040, 6824, 3040, 1013, 16216, 1

In [154]:
tokenizer.decode(example['input_ids'])

"[CLS] actiris - scrum master / gestionnaire de projets scrum master, chef de projets,. net actiris le service developpement d'actiris est compose d'une trentaine de personnes qui travaillent essentiellement sur des projets nouveaux ( dossier unique, plateforme pour la gestion des relations entre actiris et les partenaires, plateforme pour la gestion des relations entre actiris et les employeurs, mise en place d'un nouveau crm, etc. ). votre role en tant que scrum master / gestionnaire de projets, vous initiez et coordonnez, dans un environnement agile, des projets dans le cadre du developpement de nos nouvelles plateformes core business et vous veillez a leur bonne realisation dans les delais impartis. vos responsabilites vous collectez les informations necessaires au bon deroulement du projet et etes capable de discuter d'un backlog avec le metier ; vous determinez les etapes cles, planning et retro - planning highlevel sans forcement de requete aupres de l'equipe de developpement ; 

In [155]:
example = encoded_dataset['validation'][0]
print(example["labels"])


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [156]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]


['169', '170', '375']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [157]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [158]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [159]:
#raise Exception("STOP")

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [160]:
batch_size = 8
metric_name = "f1"

In [163]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"C:\\tmp\\BERT_results",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    save_steps=100
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [164]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [165]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [166]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1999,  3388,  2819,  1011,  2613,  3527, 13728,  2368,  1011,
        24296,  6112,  3992, 24296,  6112,  1999,  3388,  2819,  1011,  2613,
         3527, 13728,  2368,  2115,  3853,  2000,  2490,  1996, 27258,  3930,
         1997,  2256, 24296,  3218,  1010,  2057,  2024,  2559,  2005,  2195,
        24296,  6112,  6145,  1012,  2182,  1005,  1055,  2129,  2017,  1005,
         2222,  2191,  4254,  1024,  2005,  3469,  3934,  2017,  2147,  1999,
         2136,  2007,  2060, 24296,  6112,  6145,  1010,  6112,  5576,  8160,
         1998,  2622, 10489,  2000,  4339,  1037,  2047,  3112,  2466,  1012,
         2057,  2064,  4175,  2006,  2017,  2005,  1996,  2658,  7375,  1997,
         1996,  8518, 18011,  2000,  2017,  1012,  2005,  3760,  6112,  3934,
         2017,  2024,  1999,  3715,  2005,  1996,  2440,  8147,  1024,  2017,
        18012,  2115,  7396,  2013,  1996,  2640,  2127,  2010,  5576,  2003,
         3929,  6515,  1012,  1999,  2804,  2000, 24853,  1010, 

In [167]:
#forward pass
print(f"inputids: {type(encoded_dataset['train']['input_ids'][0])} {encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask: {type(encoded_dataset['train']['attention_mask'][0])} {encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels: {type(encoded_dataset['train'][0]['labels'])} {encoded_dataset['train'][0]['labels'].shape}")

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0),
                attention_mask=encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
                labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

inputids: <class 'torch.Tensor'> torch.Size([512])
attention_mask: <class 'torch.Tensor'> torch.Size([512])
labels: <class 'torch.Tensor'> torch.Size([42])


SequenceClassifierOutput(loss=tensor(0.7052, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0229,  0.3780, -0.2274, -0.1854,  0.1064,  0.1191,  0.1699, -0.5683,
          0.4452, -0.6845,  0.3506,  0.6934, -0.3370,  0.0500,  0.0494,  0.3355,
         -0.2689,  0.4613, -0.4316, -0.2425,  0.5372,  0.4277,  0.0832, -0.0118,
          0.3681, -0.9598,  0.2862, -0.0103,  0.1475,  0.0441,  0.1987, -0.1984,
         -0.5925, -0.2784, -0.1586, -0.0764, -0.0824, -0.3391,  0.1928,  0.2436,
          0.0190, -0.2196]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [168]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [169]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.523046,0.028986,0.493314,0.000000
2,No log,0.442285,0.000000,0.500000,0.000000
3,No log,0.400139,0.000000,0.500000,0.000000
4,No log,0.378894,0.000000,0.500000,0.000000
5,No log,0.372132,0.000000,0.500000,0.000000


TrainOutput(global_step=80, training_loss=0.4566977024078369, metrics={'train_runtime': 304.4661, 'train_samples_per_second': 2.102, 'train_steps_per_second': 0.263, 'total_flos': 168451552051200.0, 'train_loss': 0.4566977024078369, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [170]:
trainer.evaluate()

{'eval_loss': 0.5230456590652466,
 'eval_f1': 0.028985507246376812,
 'eval_roc_auc': 0.49331400307010065,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.7134,
 'eval_samples_per_second': 25.231,
 'eval_steps_per_second': 4.205,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

"Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,","142 (7),189,190,754,208,794,676,811,812,139,138"
146 (7)

In [171]:
text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [172]:
logits = outputs.logits
logits.shape

torch.Size([1, 42])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [173]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['146']
